<a href="https://colab.research.google.com/github/mrpintime/Constraints_NeuralNet/blob/main/Constraints_NeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 7  


# Approche 1  
Several Conflict Matrix

In [2]:
import random
import numpy as np

list_of_conflicts = []

while len(list_of_conflicts) < 500:
    pairs_list = set()
    matrix = np.zeros((24, 24), dtype=int)

    while matrix.sum() < 40:
        num1 = np.random.choice(range(24))
        num2 = np.random.choice(range(24))

        if num1 == num2:
            continue

        pair = (num1, num2)
        if pair in pairs_list:
            continue

        pairs_list.add(pair)
        matrix[num1, num2] = 1

    if not any(np.array_equal(matrix, conflict) for conflict in list_of_conflicts):
        list_of_conflicts.append(matrix)

In [3]:
conflicts = np.array(list_of_conflicts)

In [4]:
conflicts.shape

(500, 24, 24)

In [5]:
def create_adjacent_mask(n_seats, seats_per_row, seats_per_col):
    adjacent_mask = np.zeros((n_seats, n_seats))
    for i in range(n_seats):
        if i % seats_per_row != 0:
            adjacent_mask[i, i-1] = 1
        if i % seats_per_row != seats_per_row-1:
            adjacent_mask[i, i+1] = 1
        if i >= seats_per_row:
            adjacent_mask[i, i-seats_per_row] = 1
        if i < n_seats-seats_per_row:
            adjacent_mask[i, i+seats_per_row] = 1
    return adjacent_mask

adjacent_mask = create_adjacent_mask(24,6,4)

In [6]:
adjacent_mask.shape

(24, 24)

## Tensorflow

In [8]:
import tensorflow as tf
import numpy as np

adjacent_mask = create_adjacent_mask(24, 6, 4)

def calculate_conflict(seating_arrangement, conflict_matrix):
    ca_mul = tf.convert_to_tensor(conflict_matrix * adjacent_mask, tf.float64)
    conflicts = tf.reduce_sum(tf.matmul(tf.cast(seating_arrangement, tf.float64), ca_mul))
    return conflicts

def custom_loss(predicted_seating_arrangement, conflicts_tensor):
    loss = 0
    batch_size = predicted_seating_arrangement.shape[0]

    # Ensure the predicted seating arrangement is in float64
    predicted_seating_arrangement = tf.cast(predicted_seating_arrangement, tf.float64)

    conflict = calculate_conflict(predicted_seating_arrangement, conflicts_tensor)
    # Ensure each seat is assigned to only one person (columns should sum to 1)
    col_sum_loss = tf.reduce_sum((tf.reduce_sum(predicted_seating_arrangement, axis=1) - 1) ** 2)

    loss = col_sum_loss + conflict

    return loss / tf.cast(batch_size, tf.float64)

conflicts_tensor = tf.convert_to_tensor(conflicts, tf.float64)

model = tf.keras.Sequential([
    tf.keras.layers.Input(conflicts_tensor.shape[1:]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(24*24),
    tf.keras.layers.Reshape((24, 24)),
    tf.keras.layers.Softmax(axis=2)  # Applying softmax along the last axis
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

for epoch in range(20):
    with tf.GradientTape() as tape:
        predicted_seating_arrangement = model(conflicts_tensor, training=True)
        loss = custom_loss(predicted_seating_arrangement, conflicts_tensor)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'Epoch: {epoch}, Loss: {loss.numpy()}')

Epoch: 0, Loss: 5.570847798779119
Epoch: 1, Loss: 5.59418144604875
Epoch: 2, Loss: 5.475330373890806
Epoch: 3, Loss: 5.474966782635238
Epoch: 4, Loss: 5.4444172224305625
Epoch: 5, Loss: 5.387312345791366
Epoch: 6, Loss: 5.305951939639843
Epoch: 7, Loss: 5.212041281853652
Epoch: 8, Loss: 5.132420952923065
Epoch: 9, Loss: 5.069898444984496
Epoch: 10, Loss: 5.000904045017689
Epoch: 11, Loss: 4.947270622131727
Epoch: 12, Loss: 4.884362623486902
Epoch: 13, Loss: 4.829899376377765
Epoch: 14, Loss: 4.77549751337528
Epoch: 15, Loss: 4.733116954150112
Epoch: 16, Loss: 4.694029573080971
Epoch: 17, Loss: 4.660932104732362
Epoch: 18, Loss: 4.631501168267163
Epoch: 19, Loss: 4.6045304012002415


In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

for epoch in range(300):
    with tf.GradientTape() as tape:
        predicted_seating_arrangement = model(conflicts_tensor, training=True)
        loss = custom_loss(predicted_seating_arrangement, conflicts_tensor)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'Epoch: {epoch}, Loss: {loss.numpy()}')


Epoch: 0, Loss: 4.579114785920096
Epoch: 1, Loss: 5.893860540007823
Epoch: 2, Loss: 4.70372560090229
Epoch: 3, Loss: 4.926767641351857
Epoch: 4, Loss: 4.986821002793053
Epoch: 5, Loss: 4.916320792731844
Epoch: 6, Loss: 4.891510845305187
Epoch: 7, Loss: 4.894340311439324
Epoch: 8, Loss: 4.894858945014623
Epoch: 9, Loss: 4.883464680067839
Epoch: 10, Loss: 4.861670090463209
Epoch: 11, Loss: 4.833567498819177
Epoch: 12, Loss: 4.801749139903426
Epoch: 13, Loss: 4.767439760049987
Epoch: 14, Loss: 4.73374975494317
Epoch: 15, Loss: 4.704870889716452
Epoch: 16, Loss: 4.68214811884431
Epoch: 17, Loss: 4.665638914285193
Epoch: 18, Loss: 4.65194671578651
Epoch: 19, Loss: 4.635906267330122
Epoch: 20, Loss: 4.618849898924933
Epoch: 21, Loss: 4.601255531385126
Epoch: 22, Loss: 4.585365675824594
Epoch: 23, Loss: 4.573529364328061
Epoch: 24, Loss: 4.558374694936577
Epoch: 25, Loss: 4.542886721202614
Epoch: 26, Loss: 4.530422788371008
Epoch: 27, Loss: 4.520319560036472
Epoch: 28, Loss: 4.511526895646733

In [10]:
out = model(conflicts_tensor)
tf.argmax(out, 2)

<tf.Tensor: shape=(500, 24), dtype=int64, numpy=
array([[23,  3, 21, ..., 17, 21,  2],
       [17, 20, 21, ..., 11,  9, 19],
       [15,  6, 17, ..., 11,  6,  7],
       ...,
       [23, 23,  2, ..., 11,  2, 23],
       [23,  6, 17, ..., 17,  6, 14],
       [17, 10,  7, ..., 11,  2, 23]])>

In [11]:
tf.argmax(out, 2)[0]

<tf.Tensor: shape=(24,), dtype=int64, numpy=
array([23,  3, 21,  6,  8, 15,  5, 19, 15, 20, 10, 21, 19, 20, 13, 13, 13,
       15,  8, 15, 23, 17, 21,  2])>

In [12]:
np.unique(tf.argmax(out, 2)[0]).size

13

In [13]:
row_max = tf.reduce_max(out, axis=2)
z = tf.equal(out, tf.expand_dims(row_max, axis=2))
q = tf.cast(z, tf.float64)
row_max = tf.reduce_max(q, axis=1)
penalty_dup = tf.cast(24 * 500, tf.float64) - tf.reduce_sum(row_max)
penalty_dup = penalty_dup / tf.cast(500, tf.float64)
penalty_dup

<tf.Tensor: shape=(), dtype=float64, numpy=9.5>

In [14]:
row_max

<tf.Tensor: shape=(500, 24), dtype=float64, numpy=
array([[0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 1., 0., 1.]])>

## Pytorch

In [25]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as op

In [82]:
class SeatingArr(nn.Module):
    def __init__(self) -> None:
        super(SeatingArr, self).__init__()
        self.flatten = nn.Flatten()
        self.dense_1 = nn.Linear(24*24, 80, dtype=torch.float64)
        self.dense_2 = nn.Linear(80, 24 * 24, dtype=torch.float64)


    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.dense_1(x))
        x = F.relu(self.dense_2(x))
        x = x.view(-1,24,24)
        x = F.softmax(x, dim=2)
        return x


In [83]:
net = SeatingArr()
optim = op.Adam(params=net.parameters(), lr=0.01)

In [84]:
torch_conflicts = torch.tensor(conflicts, dtype=torch.float64)
loader = torch.utils.data.DataLoader(torch_conflicts, batch_size=64, pin_memory=True)
examples = enumerate(loader)
batch_idx, example_data = next(examples)

In [86]:
batch_idx, example_data.shape, loader.batch_size, loader.dataset.shape, len(loader.dataset), len(loader)

(0, torch.Size([64, 24, 24]), 64, torch.Size([500, 24, 24]), 500, 8)

In [87]:
example_data.type()

'torch.DoubleTensor'

In [88]:
adjacent_mask = create_adjacent_mask(n_seats=24, seats_per_row=6, seats_per_col=4)
adjacent_mask = torch.tensor(adjacent_mask, dtype=torch.float64)

def calculate_conflict(seating_arrangement, conflict_matrix):
    ca_mul = conflict_matrix * adjacent_mask
    conflicts = torch.sum(torch.matmul(seating_arrangement.type(torch.float64), ca_mul))
    return conflicts

def custom_loss(predicted_seating_arrangement, conflicts_tensor):
    loss = 0
    batch_size = predicted_seating_arrangement.shape[0]
    # Ensure the predicted seating arrangement is in float64
    predicted_seating_arrangement = predicted_seating_arrangement.type(torch.float64)

    # Calculate Conflict in produced seating arrangement
    conflict = calculate_conflict(predicted_seating_arrangement, conflicts_tensor)
    # Ensure each seat is assigned to only one person (columns should sum to 1)
    col_sum_loss = torch.sum((torch.sum(predicted_seating_arrangement, dim=1) - 1) ** 2)

    loss = col_sum_loss + conflict

    return loss / batch_size

In [89]:
# save model every 50 epochs
interval = 50

def Training(epoch):
    # train
    net.train()
    for batch_idx, conflict in enumerate(loader):
        optim.zero_grad()
        output = net(conflict)
        loss = custom_loss(output, conflict)
        loss.backward()
        optim.step()
        if epoch % interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(conflict), len(loader.dataset),
            100. * batch_idx / len(loader), loss.item()))
            # save state of model and optimizer
            torch.save(net.state_dict(), '/content/model.pth')
            torch.save(optim.state_dict(), '/content/optimizer.pth')


In [91]:
n_epoch = 100
for i in range(n_epoch):
     Training(i+1)

Train Epoch: 50 [0/500 (0%)]	Loss: 4.514548
Train Epoch: 50 [64/500 (12%)]	Loss: 4.403427
Train Epoch: 50 [128/500 (25%)]	Loss: 4.172729
Train Epoch: 50 [192/500 (38%)]	Loss: 4.910792
Train Epoch: 50 [256/500 (50%)]	Loss: 4.595813
Train Epoch: 50 [320/500 (62%)]	Loss: 3.963009
Train Epoch: 50 [384/500 (75%)]	Loss: 4.079786
Train Epoch: 50 [364/500 (88%)]	Loss: 4.517661
Train Epoch: 100 [0/500 (0%)]	Loss: 4.499898
Train Epoch: 100 [64/500 (12%)]	Loss: 4.394020
Train Epoch: 100 [128/500 (25%)]	Loss: 4.164602
Train Epoch: 100 [192/500 (38%)]	Loss: 4.891116
Train Epoch: 100 [256/500 (50%)]	Loss: 4.568378
Train Epoch: 100 [320/500 (62%)]	Loss: 3.946360
Train Epoch: 100 [384/500 (75%)]	Loss: 4.074533
Train Epoch: 100 [364/500 (88%)]	Loss: 4.522278


In [93]:
with torch.no_grad():
    output = net(torch_conflicts)
    loss = custom_loss(output, torch_conflicts)
    loss_val = loss.item()
    model_out_readable = torch.argmax(output, dim=2)

In [94]:
loss_val

4.363660095510109

In [96]:
model_out_readable[0]

tensor([14, 10, 13, 11,  9, 15, 11,  2,  7,  0,  6, 20, 13,  8, 13, 20,  0,  3,
        19, 21, 15, 17, 22,  9])

In [98]:
model_out_readable[0].view(6,4)

tensor([[14, 10, 13, 11],
        [ 9, 15, 11,  2],
        [ 7,  0,  6, 20],
        [13,  8, 13, 20],
        [ 0,  3, 19, 21],
        [15, 17, 22,  9]])